In [ ]:
%reload_ext autoreload
%autoreload 2
import colabexts
from colabexts.jcommon import *

import os, sys, datetime, re, json, importlib
from collections import defaultdict
from sys import modules
from IPython.display import HTML, Javascript
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

%matplotlib inline
pd.set_option('display.max_colwidth', 40)
pd.set_option('display.max_rows', 16)

In [ ]:
%%writefile ../imgreco.py
#!/usr/bin/env python

# DO NOT EDIT THIS FILE : Generated from notebooks/imgreco.ipynb

from mangorest.mango import webapi
import colabexts.utils as colabexts_utils
import logging, os, json, base64
import numpy as np
from PIL import Image
import io

try:
    import face_recognition
    FACE_RECOGNITION_AVAILABLE = True
except ImportError:
    FACE_RECOGNITION_AVAILABLE = False
    logging.warning("face_recognition not available. Install with: pip install face_recognition")

# Simple in-memory storage for face encodings and names
KNOWN_FACES = {}
FACE_DB_FILE = 'cogs/static/cogs/face_db.json'

def load_face_db():
    """Load face database from file"""
    global KNOWN_FACES
    try:
        if os.path.exists(FACE_DB_FILE):
            with open(FACE_DB_FILE, 'r') as f:
                data = json.load(f)
                # Convert lists back to numpy arrays
                KNOWN_FACES = {name: np.array(encoding) for name, encoding in data.items()}
            logging.info(f"Loaded {len(KNOWN_FACES)} faces from database")
    except Exception as e:
        logging.error(f"Error loading face database: {e}")
        KNOWN_FACES = {}

def save_face_db():
    """Save face database to file"""
    try:
        # Convert numpy arrays to lists for JSON serialization
        data = {name: encoding.tolist() for name, encoding in KNOWN_FACES.items()}
        with open(FACE_DB_FILE, 'w') as f:
            json.dump(data, f)
        logging.info(f"Saved {len(KNOWN_FACES)} faces to database")
    except Exception as e:
        logging.error(f"Error saving face database: {e}")

def decode_base64_image(base64_string):
    """Decode base64 image string to PIL Image"""
    try:
        # Remove data URL prefix if present
        if ',' in base64_string:
            base64_string = base64_string.split(',')[1]
        
        image_data = base64.b64decode(base64_string)
        image = Image.open(io.BytesIO(image_data))
        return np.array(image)
    except Exception as e:
        logging.error(f"Error decoding base64 image: {e}")
        return None

@webapi('/imgreco/recognize', methods=['POST'])
def recognize_face(image_data=None, **kwargs):
    """Recognize face from base64 encoded image"""
    if not FACE_RECOGNITION_AVAILABLE:
        return {'error': 'Face recognition not available. Install face_recognition library.'}
    
    if not image_data:
        return {'error': 'No image data provided'}
    
    try:
        # Load face database
        load_face_db()
        
        # Decode image
        image = decode_base64_image(image_data)
        if image is None:
            return {'error': 'Failed to decode image'}
        
        # Find face locations and encodings
        face_locations = face_recognition.face_locations(image)
        if not face_locations:
            return {'result': 'No faces found in image'}
        
        face_encodings = face_recognition.face_encodings(image, face_locations)
        
        results = []
        for face_encoding in face_encodings:
            if not KNOWN_FACES:
                results.append({'name': 'Unknown', 'message': 'No faces in database'})
                continue
            
            # Compare with known faces
            known_encodings = list(KNOWN_FACES.values())
            known_names = list(KNOWN_FACES.keys())
            
            matches = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=0.6)
            face_distances = face_recognition.face_distance(known_encodings, face_encoding)
            
            if any(matches):
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_names[best_match_index]
                    confidence = 1 - face_distances[best_match_index]
                    results.append({
                        'name': name,
                        'confidence': float(confidence),
                        'face': name,
                        'message': f'Hi {name}! (confidence: {confidence:.2f})'
                    })
                else:
                    results.append({'name': 'Unknown', 'message': 'Face not recognized'})
            else:
                results.append({'name': 'Unknown', 'message': 'Face not recognized'})
        
        return {
            'result': 'success',
            'faces_found': len(face_locations),
            'recognitions': results
        }
        
    except Exception as e:
        logging.error(f"Error in face recognition: {e}")
        return {'error': f'Face recognition failed: {str(e)}'}

@webapi('/imgreco/add_face', methods=['POST'])
def add_face(image_data=None, name=None, **kwargs):
    """Add a face to the database with a name tag"""
    if not FACE_RECOGNITION_AVAILABLE:
        return {'error': 'Face recognition not available. Install face_recognition library.'}
    
    if not image_data or not name:
        return {'error': 'Both image_data and name are required'}
    
    try:
        # Load existing database
        load_face_db()
        
        # Decode image
        image = decode_base64_image(image_data)
        if image is None:
            return {'error': 'Failed to decode image'}
        
        # Find face locations and encodings
        face_locations = face_recognition.face_locations(image)
        if not face_locations:
            return {'error': 'No faces found in image'}
        
        if len(face_locations) > 1:
            return {'error': 'Multiple faces found. Please use an image with only one face.'}
        
        face_encodings = face_recognition.face_encodings(image, face_locations)
        if not face_encodings:
            return {'error': 'Could not encode face'}
        
        # Store the face encoding
        KNOWN_FACES[name] = face_encodings[0]
        
        # Save to file
        save_face_db()
        
        return {
            'result': 'success',
            'message': f'Face for {name} added successfully',
            'total_faces': len(KNOWN_FACES)
        }
        
    except Exception as e:
        logging.error(f"Error adding face: {e}")
        return {'error': f'Failed to add face: {str(e)}'}

@webapi('/imgreco/list_faces', methods=['GET'])
def list_faces(**kwargs):
    """List all known faces in the database"""
    load_face_db()
    return {
        'result': 'success',
        'faces': list(KNOWN_FACES.keys()),
        'total_faces': len(KNOWN_FACES)
    }

# Initialize face database on module load
load_face_db()

logging.info(f"Face recognition module loaded. Available: {FACE_RECOGNITION_AVAILABLE}")

Overwriting ../imgreco.py


In [ ]:
# Test the face recognition functions
import sys
sys.path.append('../geoapp')

try:
    import imgreco
    print("Face recognition module imported successfully")
    print(f"Face recognition available: {imgreco.FACE_RECOGNITION_AVAILABLE}")
    
    # Test list faces
    result = imgreco.list_faces()
    print(f"Current faces in database: {result}")
    
except Exception as e:
    print(f"Error importing face recognition module: {e}")

In [ ]:
# Install face_recognition if not available
import subprocess
import sys

try:
    import face_recognition
    print("face_recognition is already installed")
except ImportError:
    print("Installing face_recognition...")
    print("Note: This may take a while and requires cmake and dlib")
    print("On macOS: brew install cmake")
    print("On Ubuntu: sudo apt-get install cmake")
    
    # Uncomment the line below to install face_recognition
    # subprocess.check_call([sys.executable, "-m", "pip", "install", "face_recognition"])